# Phase-Aware Speech Enhancement with Deep Complex U-Net

В данной стате рассматривается подход к очищению данных от шума при помощи модифицированной сети
U-Net - DCUnet.

Также, помимо модификации сети U-net, авторы предложили новый подход для получения маски (complex ratio mask (cRM)) и loss функцию.

## 1. Deep Complex U-Net

Для получения маски авторы предлагают использовать модифицорованную сеть U-Net.
В данной сети авторы используют комплекснозначные нейронные слои, представленные
в статье Trabelsi и др. «Deep Complex Networks».
Нейронная сеть применяется к данным, преобразованным через STFT.

Моя реализация находится в классе `src.model.dcunet.DCUnet`.

![alt text](../img/dcunet10.png )


### 1.1. Complex Convolution layer

Вместо обычных конволюционных слоев в DCUnet используется комплексные конволюции.
Определяется она следующим образом:
имеем фильтр $W = A + i B$, где $A$ и $B$ - матрицы из реальных значений. Применение данного
фильтра к вектору $h = x + i y$ определяется как $W h = (A x - B y) + i (A x + B y)$.

Моя реализация находится в классе `src.model.complexnet.ComplexConv2d`. Так же реализована деконволюция `src.model.complexnet.ComplexConvTranspose2d`.

![alt text](../img/cconv.png)

### 1.2. $\mathbb{C}\mathrm{ReLU}$

В качетве функции активации используется $\mathbb{C}\mathrm{ReLU}$:
$$
\mathbb{C}\mathrm{ReLU}(x + i y)= \mathrm{ReLU}(x) + i \mathrm{ReLU}(y).
$$

Моя реализация находится в классе `src.model.complexnet.ComplexReLU`. В реализции она представляет просто ReLU активацию.

### 1.3. Complex Batch normalization

Нормирование реализовано как перемножение 0-центрированных данных
на обратный корень матрицы ковариации 2x2:
$$
\tilde{x} = (V)^{-\frac{1}{2}} (x - \mathbb{E}(x))
$$
где
$$
V = \begin{pmatrix}
V_{rr} & V_{ri} \\
V_{ir} & V_{ii}
\end{pmatrix}
$$

В реализации имеем $V_{\mathrm{run}}$, $E_{\mathrm{run}}$
для нормализации при вычислении тестовых данных. $V_{ii}$ и $V_{rr}$ инициализируются как
$\frac{1}{\sqrt{2}}$. Во время тренировки они сдвигаются в сторону полученных V и E на тестовых данных
с параметром magnitude.

Так же как и в обычной batch нормализации используется два обучающих параметра $\beta \in \mathbb{C}$ 
и $\gamma$ - матрица 2x2. Тогда комплексная batch нормализация определяется как:
$$
\mathrm{BN} = \gamma x + \beta
$$

Начальные значения для параметров имеем следующие: $\gamma_{ii}$ и $\gamma_{rr}$ как $\frac{1}{\sqrt{2}}$,
$\beta = 0 + i 0$.

Моя реализация находится в классе `src.model.complexnet.ComplexBatchNorm2d`.

## 2. cRM

![alt text](../img/crm.png)

Оцененная речевая спектрограмма вычисляется умножением
оценочной маски $M_{t,f}$ на входной спектрограмме $X_{t,f}$ следующим образом:
$$
\hat{Y}_{t,f}  = \hat{M}_{t,f}  \cdot \hat{X}_{t,f}
$$

Для получения маски авторы предлагают подход, основанный на полярных координатах. 
Маска получается следующим образом:
$$
\hat{M}_{t,f} = |\hat{M}_{t,f}| \cdot e^{i \theta_{\hat{M}_{t,f}}} = \hat{M}^{\mathrm{mag}}_{t,f} \cdot \hat{M}^{\mathrm{phase}}_{t,f}
$$
$$
\hat{M}^{\mathrm{phase}}_{t,f} = \frac{ O_{t,f} }{ |O_{t,f}|}
$$
$$
\hat{M}^{\mathrm{mag}}_{t,f} = \tanh{( |O_{t,f}|)}
$$
где $O_{t,f}$ - выход сети.

## 3. Loss функция

Обычно в качестве loss функции используется MSE между чистым и предсказаным значением в STFT пространстве.
Но в таком случе сигнал имеет не очень хорошую структуру.
Авторы предлагают следующую функцию, которая учитывает соотношение шума и голоса, и используют только сигналы без
STFT преобразования:
$$
\mathrm{loss}_{wSDR}(x, y \bar{y}) := \alpha \mathrm{loss}_{SDR}(y, \bar{y}) +(1- \alpha)\mathrm{loss}_{SDR}(z, \bar{z}) 
$$
где $z$ - исходный шум, $\bar{z} = x - \bar{y}$,
$$
\mathrm{loss}_{SDR}(y \bar{y}) := -\frac{<y, \bar{y}>}{||y|| ||\bar{y}||}
$$
и
$$
\alpha = \frac{||y||}{||y|| + ||z||}
$$


## 3. Система устранения шума

В итоге можно собрать все в одну систему для устраненя шума. Сначала зашумленные данные
преобразуются при помощи STFT. Далее проходят через DCUnet и при помощи cRM получается маска и
выходное представление. Полученное представление преобразуется через ISTFT и получается 
звук без шума.


Моя реализация находится в классе `src.model.enhmodel.EnhModel`.


![alt text](../img/enh.png)

# Результаты

## Авторы
Авторы статьи получили следующие результаты в сравнении с другими моделями:
![alt text](../img/results.png)

## Мои эксперемент

В качестве данных для чистого голоса была взята выборка из набора данных LIBRISPEECH, а в качестве шума 
использовались записи из набора данных DEMAND.

Все файлы имеют sample rate 42kHz. Для обучение он был уменьшен до 16kHz.

Зашумление происходило так: чистый голос был объединён с 4-мя шумами с соотношением сигнал-шум (SNR) 15, 10, 5 или 0 dB.

В качестве тренировочной выборки использовались 50 спикеров и 8 типов шумов.
В качестве тестовой выборки использовались 5 спикеров и 2 типа шума.

В каждой эпохе обучалось по $20 \cdot 10$ случайных зашумлённых голосов и $10$ тестировалось.


***Примечание:*** из-за нехватки вычеслительных ресурсов я провел эксперемент только на данных DCUnet10 и не смог провести достаточно вычислений для получения результата, сравнимого с результатами авторов, но прогресс обучения прослеживается. Также наверное я взял слишком много тестовых и тренеровочных данных.

![alt text](../img/train_history.png)

**Значение PESQ:** 2.3644541573524473

# Выводы

Была реализована система отделения шума в речи на основе DCUnet.

Проверена работоспособность и проведён эксперимент.

Сложно сказать о качестве модели, основываясь на моих результатах.

В дальнейшем возможно улучшить данную модель, например, использовав resnet блоки вместо U-net, или 
попробовать повысить робастность выходного слоя при помощи дополнительных регуляризаций.

На практике данная модель может применяться в улучшении качества аудиосвязи при плохом качестве передачи
или при разговоре в зашумлённых помещениях.